In [9]:
import json 
import unicodedata

def normalize(s: str):
    return unicodedata.normalize("NFC", s)

f = open("./data/GSM8K/train.jsonl", "r")
data = []
for line in f:
    line = json.loads(line)
    line["question"] = normalize(line["question"])
    answer = normalize(line["answer"]).split("####")[-1]
    line["answer"] = float(answer.replace(",", "").strip())
    data.append(line)

f.close()

f = open("./data/GSM8K/test.jsonl", "r")
for line in f:
    line = json.loads(line)
    line["question"] = normalize(line["question"])
    answer = normalize(line["answer"]).split("####")[-1]
    line["answer"] = float(answer.replace(",", "").strip())
    data.append(line)

f.close()

In [10]:
len(data)

8792

In [11]:
f = open("./data/GSM8K/gsm6k_code.jsonl", "r")
code_data = []
for line in f:
    line = json.loads(line)
    line["question"] = normalize(line["question"])
    code_data.append(line)

f.close()

f = open("./data/GSM8K/gsm3k_code.jsonl", "r")
for line in f:
    line = json.loads(line)
    line["question"] = normalize(line["question"])
    code_data.append(line)

f.close()

final_data = []
for s1 in code_data:
    new_sample = {
        "question": s1["question"],
        "instruction": s1["instruction"],
        "code": s1["code"],
        "answer": None
    }
    for s2 in data:
        if s1["question"] == s2["question"]:
            new_sample["answer"] = s2["answer"]
            break 
    final_data.append(new_sample)

for i, s in enumerate(final_data):
    if s["answer"] is None:
        print(i)


In [15]:
len(final_data)

8929

In [17]:
print((final_data[0]["code"])[9:-3])


# Define the number of clips sold in April
clips_april = 48

# Divide the number of clips sold in April by 2 to calculate the number of clips sold in May
clips_may = clips_april / 2

# Add the number of clips sold in April and the number of clips sold in May to calculate the total number of clips sold in April and May
clips_total = clips_april + clips_may

# Print the total number of clips sold in April and May
print(clips_total)



In [18]:
from src.utils.python_executor import PythonExecutor
import re

CODE_PATTERN = re.compile(r"```python([\s\S]*)```")

def extract_code(s: str):
    return CODE_PATTERN.findall(s)[0]

executor = PythonExecutor(get_answer_from_stdout=True)

In [19]:
from tqdm import  tqdm
final_data_1 = []
for s in tqdm(final_data):
    try:
        code = extract_code(s["code"])
        predictions = float(executor.apply(code)[0])
    except:
        continue
    if abs(predictions - s["answer"]) < 1e-5:
        final_data_1.append(s)

100%|██████████| 8929/8929 [16:46<00:00,  8.87it/s]


In [20]:
len(final_data_1)

6648

In [72]:
with open("./data/GSM8K/gsm8k_code.jsonl", "a") as f:
    for s in final_data_1:
        d = json.dumps(s, ensure_ascii=False)+"\n"
        f.write(d)

In [73]:
final_data_1[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'instruction': 'STEP\nStep 1: The number of clips sold in April is 48.\nStep 2: Divide the number of clips sold in April by 2 to calculate the number of clips sold in May.\nStep 3: Add the number of clips sold in April and the number of clips sold in May to calculate the total number of clips sold in April and May.',
 'code': '```python\n# Define the number of clips sold in April\nclips_april = 48\n\n# Divide the number of clips sold in April by 2 to calculate the number of clips sold in May\nclips_may = clips_april / 2\n\n# Add the number of clips sold in April and the number of clips sold in May to calculate the total number of clips sold in April and May\nclips_total = clips_april + clips_may\n\n# Print the total number of clips sold in April and May\nprint(clips_total)\n```',
 'answer': 72.0}

In [76]:
sample

{'question': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?',
 'instruction': 'STEP\nStep 1: The amount Weng earns per hour babysitting is $12.\nStep 2: Calculate how much Weng earns per minute babysitting by dividing the amount Weng earns per hour by 60. In this case, $12 divided by 60 equals $0.20 per minute.\nStep 3: Multiply the result in step 2 by 50 to find the amount of money Weng earns after 50 minutes of babysitting. In this case, $0.20 multiplied by 50 equals $10.',
 'code': '```python\n# Define the amount Weng earns per hour babysitting\nhourly_rate = 12\n\n# Calculate how much Weng earns per minute babysitting by dividing the hourly rate by 60\nminute_rate = hourly_rate / 60\n\n# Multiply the minute rate by 50 to find the amount of money Weng earns after 50 minutes of babysitting\nearnings = minute_rate * 50\n\n# Print the earnings\nprint(earnings)\n```',
 'answer': 10.0}

In [21]:
from tqdm import  tqdm
with open("./data/gsm8k_zalo/gsm8k_v1.jsonl", "a") as f:
    for sample in tqdm(final_data_1):
        code = f"```python\n{extract_code(sample['code'])}\n```"
        s = {
            "messages": [
                {
                    "role": "user",
                    "content": f"Solve the following problem: {sample['question']}"
                },
                {
                    "role": "assistant",
                    "content": (
                        "Let's break it down step by step first:\n"
                        f"{sample['instruction']}\n"
                        "Here's the Python code based on the plan above:\n"
                        f"{code}"    
                    )
                }
            ]
        }
        d = json.dumps(s, ensure_ascii=False)+"\n"
        f.write(d)


100%|██████████| 6648/6648 [00:00<00:00, 22682.20it/s]


In [1]:
import pandas as pd
import json
import unicodedata

def normalize(s: str):
    return unicodedata.normalize("NFC", s)

    
f  = open("./data/zalo/train/math_code_v2.jsonl")
code_data = []
for line in f:
    line = json.loads(line.strip())
    code_data.append(line)
f.close()

with open("./data/zalo/train/math_train.json", "r") as f:
    original_data = json.loads(f.read())

for sample in code_data:
    sample["answer"] = None
    for sample_1 in original_data["data"]:
        question = normalize(sample["question"].strip())
        question_1 = normalize(sample_1["question"].strip())
        if question_1[:-1] in question:
            sample["answer"] = normalize(sample_1["answer"])

In [2]:
len(code_data)

1208

In [3]:
from tqdm import  tqdm
import re
from src.utils.python_executor import PythonExecutor


CODE_PATTERN = re.compile(r"```python([\s\S]*)```")

def extract_code(s: str):
    return CODE_PATTERN.findall(s)[0]

executor = PythonExecutor(get_answer_from_stdout=True)

def is_float(string: str):
    try:
        float(string)
        return True
    except ValueError:
        return False


COMPARISON_PATTERN = re.compile("if ([\w]+==.*|[\w]+ ==.*):")


def fix_rounding_error(code: str):
    comparisons = COMPARISON_PATTERN.findall(code)
    if len(comparisons) == 0:
        return code
    answer_variable = comparisons[0].split("==")[0]
    answer_value = executor.apply(
        code + f"\nprint({answer_variable})"
    )[0]
    if is_float(answer_value):
        for comparison in comparisons:
            choice_variable = comparison.split("==")[-1]
            str_to_replace = f"abs({answer_variable}-{choice_variable}) < 1e-8"
            code = code.replace(comparison, str_to_replace)
    
    return code

def execute_python_code(code: str):
    output = executor.apply(code)
    if output[1] == 'Done':
        if output[0] == '':
            return code, "Missing print function!!!!"
        if output[0].strip() not in ["A", "B", "C", "D", "E"]:
            code = fix_rounding_error(code)
            output = executor.apply(code)        

    return code, output[0]

In [4]:
code_data_1 = []
drop_ids = []
for i, s in enumerate(tqdm(code_data)):
    try:
        code = extract_code(s["code"])
    except:
        drop_ids.append(i)
        continue
    tmp = execute_python_code(code)
    _, output = tmp
    if output.strip().lower() == s["answer"][0].lower():
        code_data_1.append(s)
    else:
        drop_ids.append(i)

100%|██████████| 1208/1208 [03:08<00:00,  6.40it/s]


In [5]:
len(code_data_1)

785

In [6]:
code_data_1[0]

{'question': 'Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính?\nA. 80,000 : 6\nB. 80,000 x 6\nC. 80,000 : (6 x 100)\nD. (80,000 x 6) : 100\n',
 'instruction': 'STEP\nStep 1: The capital invested is 80,000 VND, and the loss percentage is 6%.\nStep 2: To calculate the loss amount, multiply the initial capital amount (80,000) by the loss percentage (6%).\nStep 3: Compare the calculated results with each answer to choose the correct answer.',
 'code': 'I can try to write a Python code based on the instructions you gave me. Here is what I came up with:\n\n```python\ninitial_capital = 80000\nloss_percentage = 6 / 100\n\nloss = initial_capital * loss_percentage\n\nanswers = {\n    "A": 80000 / 6,\n    "B": 80000 * 6,\n    "C": 80000 / (6 * 100),\n    "D": (80000 * 6) / 100\n}\nif loss == answers["A"]:\n    answer = "A"\nelif loss == answers["B"]:\n    answer = "B"\nelif loss == answers["C"]:\n    answer = "C"\nelif loss == answers["D"]:\n    answer =

In [8]:
from tqdm import  tqdm
import json

from src.utils.utils import add_notes

with open("./data/gsm8k_zalo/zalo_v1.jsonl", "a") as f:
    for sample in tqdm(code_data_1):
        code = f"```python\n{extract_code(sample['code'])}\n```"
        question = add_notes(sample['question'])
        s = {
            "messages": [
                {
                    "role": "user",
                    "content": f"Solve the following multiple-choices problem: {question}"
                },
                {
                    "role": "assistant",
                    "content": (
                        "Let's break it down step by step first:\n"
                        f"{sample['instruction']}\n"
                        "Here's the Python code based on the plan above:\n"
                        f"{code}"    
                    )
                }
            ]
        }
        d = json.dumps(s, ensure_ascii=False)+"\n"
        f.write(d)

100%|██████████| 785/785 [00:00<00:00, 16837.79it/s]


In [75]:
print(code_data[5]["instruction"])

STEP
Step 1: Convert 10% to decimal form by dividing it by 100.
Step 2: Multiply the decimal form of 10% by 5 dm to find the answer.
Step 3: Compare the calculated result with each answer to choose the correct option.


In [71]:
drop_ids

[5,
 7,
 18,
 20,
 24,
 31,
 36,
 39,
 40,
 43,
 45,
 51,
 53,
 57,
 66,
 69,
 73,
 85,
 86,
 89,
 90,
 91,
 93,
 95,
 97,
 99,
 101,
 104,
 110,
 119,
 131,
 134,
 137,
 149,
 150,
 154,
 158,
 165,
 166,
 171,
 174,
 181,
 182,
 190,
 197,
 199,
 202,
 203,
 204,
 205,
 206,
 207,
 216,
 229,
 233,
 234,
 242,
 248,
 249,
 255,
 256,
 257,
 260,
 263,
 264,
 267,
 268,
 269,
 271,
 275,
 277,
 279,
 280,
 281,
 285,
 289,
 290,
 291,
 293,
 296,
 297,
 299,
 302,
 307,
 310,
 311,
 316,
 317,
 318,
 321,
 322,
 324,
 325,
 326,
 327,
 329,
 330,
 332,
 334,
 335,
 336,
 337,
 342,
 346,
 348,
 350,
 353,
 354,
 355,
 360,
 361,
 363,
 364,
 374,
 383,
 384,
 387,
 389,
 395,
 396,
 399,
 407,
 413,
 414,
 415,
 416,
 421,
 429,
 431,
 436,
 437,
 438,
 440,
 442,
 447,
 452,
 453,
 459,
 460,
 465,
 469,
 471,
 476,
 477,
 478,
 480,
 482,
 483,
 484,
 485,
 486,
 491,
 492,
 494,
 497,
 498,
 499,
 500,
 503,
 504,
 505,
 507,
 509,
 510,
 511,
 514,
 515,
 517,
 518,
 519,
 520,
 5

In [27]:
add_id = [5, 7, 36, 43, 53, 66, 69, 73, 85, 86, 89, 90, 93, 97, 99, 101, 104, 110, 119]
correct_id = [91]

In [78]:
ind = 1205
print(code_data[ind]["question"])
print(code_data[ind]["instruction"])
print(code_data[ind]["answer"])

Một ô tô đi từ Hà Nội lúc 11 giờ trưa và đến Vinh lúc 5 giờ 30 phút chiều. Dọc đường ô tô dừng ở Ninh Bình và Thanh Hóa mỗi nơi 15 phút. Hỏi không kể thời gian dừng dọc đường, ô tô đi hết quãng đường từ Hà Nội đến Vinh mất bao nhiêu thời gian ?
A. 4 giờ 30 phút
B. 6 giờ 30 phút
C. 6 giờ 15 phút
D. 6 giờ

STEP
Step 1: Calculate the total time spent on the road by subtracting the time of departure from the time of arrival. 
Step 2: Subtract the time spent at each stop (Ninh Binh and Thanh Hoa) from the total time calculated in step 1. 
Step 3: Compare the results with the answers to find the correct option.
D. 6 giờ


In [79]:
print(code_data[ind]["code"])

CODE
```python
departure_time = 11  # in hours
arrival_time = 5 + 30/60  # in hours

total_time_on_road = arrival_time - departure_time

# Time spent at each stop in hours
stop_time = 15 / 60

# Subtract stop time from the total time on the road
total_time_without_stops = total_time_on_road - (2 * stop_time)

answers = {
    "A": 4 + 30/60,
    "B": 6 + 30/60,
    "C": 6 + 15/60,
    "D": 6
}
if total_time_without_stops == answers["A"]:
    answer = "A"
elif total_time_without_stops == answers["B"]:
    answer = "B"
elif total_time_without_stops == answers["C"]:
    answer = "C"
elif total_time_without_stops == answers["D"]:
    answer = "D"
else:
    answer = "No answer"

print(answer)
```

This code calculates the total time spent on the road without considering the time spent at stops in Ninh Binh and Thanh Hoa. It then compares this result with the provided answer choices to determine the correct option.


In [74]:
x = (27 * 8) / 3

answers = {
    "A": 56,
    "B": 64,
    "C": 72,
    "D": 80
}
if x == answers["A"]:
    answer = "A"
elif x == answers["B"]:
    answer = "B"
elif x == answers["C"]:
    answer = "C"
elif x == answers["D"]:
    answer = "D"
else:
    answer = "No answer"

print(answer)

C


In [69]:
remainder

9

In [57]:
volume_m3

2.014

In [60]:
code = """
percentage = 0.1
area = 11.5

result = percentage * area

answers = {
    "A": 10.15,
    "B": 1.5,
    "C": 15.5,
    "D": 1.15
}
if result == answers["A"]:
    answer = "A"
elif result == answers["B"]:
    answer = "B"
elif result == answers["C"]:
    answer = "C"
elif result == answers["D"]:
    answer = "D"
else:
    answer = "No answer"

print(answer)
"""
def fix_rounding_error(code: str):
    comparisons = COMPARISON_PATTERN.findall(code)
    if len(comparisons) == 0:
        return code
    answer_variable = comparisons[0].split("==")[0]
    answer_value = executor.apply(
        code + f"\nprint({answer_variable})"
    )[0]
    if is_float(answer_value):
        for comparison in comparisons:
            choice_variable = comparison.split("==")[-1]
            str_to_replace = f"abs({answer_variable}-{choice_variable}) < 1e-8"
            code = code.replace(comparison, str_to_replace)
    
    return code

def execute_python_code(code: str):
    output = executor.apply(code)
    if output[1] == 'Done':
        if output[0].strip() not in ["A", "B", "C", "D", "E"]:
            code = fix_rounding_error(code)
            output = executor.apply(code)
    else:
        if output[0] == '':
            return "Missing print function!!!!"

    return output[0], code

In [62]:
print(execute_python_code(code)[1])

Hellllllllll

percentage = 0.1
area = 11.5

result = percentage * area

answers = {
    "A": 10.15,
    "B": 1.5,
    "C": 15.5,
    "D": 1.15
}
if abs(result - answers["A"]) < 1e-8:
    answer = "A"
elif abs(result - answers["B"]) < 1e-8:
    answer = "B"
elif abs(result - answers["C"]) < 1e-8:
    answer = "C"
elif abs(result - answers["D"]) < 1e-8:
    answer = "D"
else:
    answer = "No answer"

print(answer)



In [54]:
answer_variable

NameError: name 'answer_variable' is not defined

In [57]:
executor.apply(
    code + f"\nprint(result )"
)[0]

'1.1500000000000001\n'

In [45]:
percentage = 0.1
area = 11.5

result = percentage * area

answers = {
    "A": 10.15,
    "B": 1.5,
    "C": 15.5,
    "D": 1.15
}
if result == answers["A"]:
    answer = "A"
elif result == answers["B"]:
    answer = "B"
elif result == answers["C"]:
    answer = "C"
elif result == answers["D"]:
    answer = "D"
else:
    answer = "No answer"

print(answer)

print(result )

No answer
1.1500000000000001


In [48]:
code

'python\nfirst_addend = 23\nsecond_addend = first_addend + 12\n\nsum = first_addend + second_addend\n\nanswers = {\n    "A": 35,\n    "B": 47,\n    "C": 11,\n    "D": 58\n}\nif sum == answers["A"]:\n    answer = "A"\nelif sum == answers["B"]:\n    answer = "B"\nelif sum == answers["C"]:\n    answer = "C"\nelif sum == answers["D"]:\n    answer = "D"\nelse:\n    answer = "No answer"\n\nprint(answer)\n'

In [212]:
print(original_data["data"][1]["question"][:-1])

8 dm2 24 cm2 = ……… dm2. Số thích hợp điền vào chỗ chấm là


In [188]:
code_data[0]["question"]

'Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính?\nA. 80,000 : 6\nB. 80,000 x 6\nC. 80,000 : (6 x 100)\nD. (80,000 x 6) : 100\n'

In [82]:
print(line["code"])

I will try to write a Python code based on your plan. Here is what I came up with:

```python
initial_capital = 80000
loss_percentage = 6 / 100

loss = initial_capital * loss_percentage

answers = {
    "A": 80000 / 6,
    "B": 80000 * 6,
    "C": 80000 / (6 * 100),
    "D": (80000 * 6) / 100
}
if loss == answers["A"]:
    answer = "A"
elif loss == answers["B"]:
    answer = "B"
elif loss == answers["C"]:
    answer = "C"
elif loss == answers["D"]:
    answer = "D"
else:
    answer = "No answer"

print(answer)
```

The result of running this code is **D**. This means that the correct answer to the multiple-choices problem is **D**. To calculate the loss amount, we have to multiply the initial capital amount by the loss percentage and divide by 100. This is equivalent to `(80,000 x 6) : 100`.


In [7]:
c = """
a = 20.5
b = 19.6

loss = a - b

answers = {
    "A": 0.9,
    "B": 1,
    "C": 1.1,
    "D": 0.5
}
if loss== answers["A"]:
    answer = "A"
elif loss == answers["B"]:
    answer = "B"
elif loss == answers["C"]:
    answer = "C"
elif loss == answers["D"]:
    answer = "D"
else:
    answer = "No answer"

print(answer)
"""

In [174]:
if executor.apply(c)[0] == "No answer\n":
    new_c = fix_rounding_error(c)
    answer = executor.apply(new_c)[0]

In [175]:
answer

'A\n'

In [166]:
def is_float(string: str):
    try:
        float(string)
        return True
    except ValueError:
        return False

comparisons = comparison_pattern.findall(c)
answer_variable = comparisons[0].split("==")[0]
new_c = c + f"\nprint({answer_variable})"
answer_value = executor.apply(new_c)[0]
if is_float(answer_value):
    for comparison in comparisons:
        choice_variable = comparison.split("==")[-1]
        str_to_replace = f"abs({answer_variable}-{choice_variable}) < 1e-8"
        c = c.replace(comparison, str_to_replace)

In [167]:
print(c)


a = 20.5
b = 19.6

loss = a - b

answers = {
    "A": 0.9,
    "B": 1,
    "C": 1.1,
    "D": 0.5
}
if abs(loss- answers["A"]) < 1e-8:
    answer = "A"
elif abs(loss- answers["B"]) < 1e-8:
    answer = "B"
elif abs(loss- answers["C"]) < 1e-8:
    answer = "C"
elif abs(loss- answers["D"]) < 1e-8:
    answer = "D"
else:
    answer = "No answer"

print(answer)



In [168]:
executor.apply(c)[0]

'A\n'

In [141]:
comparisons

['loss == answers["A"]',
 'loss == answers["B"]',
 'loss == answers["C"]',
 'loss == answers["D"]']

In [152]:
a = 20.5
b = 19.6

loss = a - b

In [153]:
loss

0.8999999999999986